# Rent Division

In the paper [Which Is the Fairest (Rent Division) of Them All?
](http://procaccia.info/papers/rent.pdf), it was suggested that the popular envy-free model is insufficient. Below we presnet an initial implmentation of the envy-free model.

## Variables

* Let $x_{ij}$ be the binary variable that suggests assignment of agent $i$ for room $j$
* Let $p_{ij}$ be the continuous variable that suggests payment of agent $i$ for room $j$

Above are the varaibles we want to know at the end of model optimization. Furthermore, we have constant variables $v_{ij}$ that stands for the private evaluation of room $j$ for agent $i$.

In [1]:
from gurobipy import Model, GRB, quicksum
import numpy as np

# create a LP model
model = Model()

# input parameters
num_agents = 3
v = np.array([
    [3, 0, 0],
    [0, 3, 0],
    [0, 0, 3]    
])
rent = v[0].sum()

assert(len(v) == num_agents)

# variables
n = range(3)
x = {}
p = {}
for i in n:
    for j in n:
        x[str([i, j])] = model.addVar(
            vtype=GRB.BINARY, 
            name=str([i, j])
        )
        p[str([i, j])] = model.addVar(
            vtype=GRB.CONTINUOUS, 
            name=str([i, j])
        )

model.update()

print(x)

Academic license - for non-commercial use only
{'[0, 0]': <gurobi.Var [0, 0]>, '[0, 1]': <gurobi.Var [0, 1]>, '[0, 2]': <gurobi.Var [0, 2]>, '[1, 0]': <gurobi.Var [1, 0]>, '[1, 1]': <gurobi.Var [1, 1]>, '[1, 2]': <gurobi.Var [1, 2]>, '[2, 0]': <gurobi.Var [2, 0]>, '[2, 1]': <gurobi.Var [2, 1]>, '[2, 2]': <gurobi.Var [2, 2]>}


## Objective

We want to maximize the social welfare

$$ \max \sum x_{ij} v_{ij}$$


In [2]:

# objective
welfare = []
for i in n:
    for j in n:
        welfare += [x[str([i, j])] * v[i,j]]
model.setObjective(
    quicksum(welfare),
    GRB.MAXIMIZE
)


## Constraints

* $\sum x_ij = 1, \forall i$ (Each agent can be assigned exactly one room)
* $\sum p_ij = rent, \forall i,j$ (Agent's payments sum up to rent)
* $(v_{ij} - p_{ij}) x_{ij} \geq (v_{ij'} - p_{ij'}) x_{ij}, j' \in N-\{j\}, j \in N, i \in N$ (Agent's utility for the selected room is at least as high as the utility for any other room)

In [3]:
# constraints
c = {}
# c1: payment must add up to rent
c["rent"]  = model.addConstr(
    quicksum(p[str([i, j])] for i in n for j in n) == rent, 
    name="rent"
)
for i in n:
    # c2: each agent must be assigned exactly one room
    c[str(['x', i, j, 1])] = model.addConstr(
        quicksum(x[str([i, j])] for j in n) == 1, 
        name=str(['x', i, j, 1])
    )

    for j in n:
        # c3: all payments are nonnegative
        c[str(['p', i, j, 0])] = model.addConstr(
            p[str([i, j])] >= 0, 
            name=str(['p', i, j, 0])
        )

        # c4: non-envy
        non_j = [jp for jp in n if jp !=j] # every other room
        for jp in non_j:
            model.addConstr(
                (v[i,j] - p[str([i, j])]) * x[str([i, j])] >= (v[i,jp] - p[str([i, jp])]) * x[str([i, j])], 
                name=str(['u', i, j, 'u_2', i, jp])
            )
model.update()
model.optimize()
print()
print("*****Thus the objective value is " + str(model.ObjVal))

Optimize a model with 13 rows, 18 columns and 27 nonzeros
Model has 18 quadratic constraints
Variable types: 9 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [3e+00, 3e+00]
  Objective range  [3e+00, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 9 rows and 0 columns
Presolve time: 0.01s
Presolved: 67 rows, 45 columns, 165 nonzeros
Variable types: 36 continuous, 9 integer (9 binary)

Root relaxation: objective -9.000000e+00, 0 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       9.0000000    9.00000  0.00%     -    0s

Explored 1 nodes (0 simplex iterations) in 0.06 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 9 

Optimal solution found (tolerance 1.00e-04)
Best

Now let's see the allocation

In [4]:
print(x)

{'[0, 0]': <gurobi.Var [0, 0] (value 1.0)>, '[0, 1]': <gurobi.Var [0, 1] (value -0.0)>, '[0, 2]': <gurobi.Var [0, 2] (value -0.0)>, '[1, 0]': <gurobi.Var [1, 0] (value -0.0)>, '[1, 1]': <gurobi.Var [1, 1] (value 1.0)>, '[1, 2]': <gurobi.Var [1, 2] (value -0.0)>, '[2, 0]': <gurobi.Var [2, 0] (value -0.0)>, '[2, 1]': <gurobi.Var [2, 1] (value -0.0)>, '[2, 2]': <gurobi.Var [2, 2] (value 1.0)>}


And the payment

In [6]:
print(p)

{'[0, 0]': <gurobi.Var [0, 0] (value 0.0)>, '[0, 1]': <gurobi.Var [0, 1] (value 3.0)>, '[0, 2]': <gurobi.Var [0, 2] (value 0.0)>, '[1, 0]': <gurobi.Var [1, 0] (value 0.0)>, '[1, 1]': <gurobi.Var [1, 1] (value 0.0)>, '[1, 2]': <gurobi.Var [1, 2] (value 0.0)>, '[2, 0]': <gurobi.Var [2, 0] (value 0.0)>, '[2, 1]': <gurobi.Var [2, 1] (value 0.0)>, '[2, 2]': <gurobi.Var [2, 2] (value 0.0)>}
